# Project 2 - Distributed Memory Sorting

**Due November 14, 2019**

Same logistic rules as Project 1 apply, regarding

This notebook can be developed on a single core of any type, but will be graded on 4 nodes with 28 cores per node

In [1]:
module use $CSE6230_DIR/modulefiles
module unload cse6230/core
module load cse6230/gcc-omp-gpu

|                                                                         |
|       A note about python/3.6:                                          |
|       PACE is lacking the staff to install all of the python 3          |
|       modules, but we do maintain an anaconda distribution for          |
|       both python 2 and python 3. As conda significantly reduces        |
|       the overhead with package management, we would much prefer        |
|       to maintain python 3 through anaconda.                            |
|                                                                         |
|       All pace installed modules are visible via the module avail       |
|       command.                                                          |
|                                                                         |


## Objectives

- The goal of this project is to *use profiling* to optimize the performance of
  an MPI-based library for distributed memory sorting.
  - The main library interface is (as declared in [proj2sorter.h](proj2sorter.h)):

``` C
    /* This is the default implementation of sorting:
     * \param[in] sorter       The sorting context.  Put all of your customizations
     *                         in this object.  Defined in proj2sorter_impl.h, where
     *                         you can change the struct to include more data
     * \param[in] numKeysLocal The number of keys on this process.
     * \param[in] uniform      True if there are the same number of keys on each process
     * \param[in/out] keys     The input array.  On output, should be globally
     *                         sorted in ascending order.
     * \return                 Non-zero if an error occured.
     */
    int Proj2SorterSort(Proj2Sorter sorter, size_t numKeysLocal, int uniform, uint64_t *keys);
```
  - The small library comes with some logging and error macros (see
      [proj2.h](proj2.h)) as well as an interface for obtaining/restoring workspace
      arrays (see `Proj2SorterGetWorkArray()` and `Proj2SorterRestoreWorkArray()` in
      [proj2sorter.h](proj2sorter.h).  To be memory neutral, restore every
      workspace that you get.

  - Functioning parallel implementations have been provided, one based on
      [quicksort](https://en.wikipedia.org/wiki/Quicksort#Parallelization) and
      one base on [bitonic mergesort](https://en.wikipedia.org/wiki/Bitonic_sorter).

  - A [template library](https://github.com/swenson/sort) originated by Chris
      Swenson has been imported for a quicksort implementation that is faster
      than `qsort` from the standard library.  The template library includes
      other implementations that you are welcome to explore.
      
      If you go through the work of bringing in a serial sorting library, you're welcome to use it,
      as long as you put it somewhere the TA and I can access it.

  - Indeed, as with previous assignments, the implementation details are up
      to you.  There is a test program
      ([test_proj2.c](test_proj2.c)), which may not be edited, that calls your
      library.  It will test the sorting bandwidth (bytes sorted per second) of
      your code on random data at varying numbers of *keys
      per MPI process* (a *key* in our library is just a `uint64_t`: a large
      integer).  You may incorporate additional files into your library by
      adding a `Makefile.inc` file to your project.

```
    ./test_proj2 MIN_KEYS_PER_PROCESS MAX_KEYS_PER_PROCESS MULTIPLIER SEED NUM_REPS UNIFORM_SIZE UNIFORM_KEYS PARTIALLY_SORTED
```

This means that the test program seeds the random number generator with `SEED`,
starts with `MIN_KEYS_PER_PROCESS`, tests `NUM_REPS` times to get an average,
and gets the next problem size by multiplying by `MULTIPLIER`, until at most
`MAX_KEYS_PER_PROCESS`.  If `UNIFORM_SIZE` is `0`, then the number of keys per process will vary between `MIN_KEYS_PER_PROCESS` and `2*MIN_KEYS_PER_PROCESS`. IF `UNIFORM_KEYS` is `0`, each process will have a keys in its own randomly chosen interval.  If `PARTIALLY_SORTED` is `1`, the input array is random but generally increasing; if `PARTIALLY_SORTED` is `-1` the input array is random but generally decreasing.  In your testing, you will probably want to test **one
problem size** at a time.  If you are
having problems with correctness (segmentation faults, hangs/deadlocks, etc.),
it is best to work those out on your workstation/laptop is possible before
using SUs on Stampede2.  You are starting (knock on wood) from a correct
implementation: try to work in small changes, testing for correctness at each change.

In [51]:
make clean
make test_proj2

rm -f libproj2.so  proj2.o proj2sorter.o local.o bitonic.o quicksort.o test_proj2.o test_proj2
mpicc -I../../utils/Random123/include -g -Wall -std=c99 -fopenmp -fpic -O3 -c -o test_proj2.o test_proj2.c
mpicc -I../../utils/Random123/include -g -Wall -std=c99 -fopenmp -fpic -O3 -c -o proj2.o proj2.c
mpicc -I../../utils/Random123/include -g -Wall -std=c99 -fopenmp -fpic -O3 -c -o proj2sorter.o proj2sorter.c
mpicc -I../../utils/Random123/include -g -Wall -std=c99 -fopenmp -fpic -O3 -c -o local.o local.c
swensonsort/sort.h:1432: warning: ‘proj2_swenson_sqrt_sort’ defined but not used
swensonsort/sort.h:2166: warning: ‘proj2_swenson_grail_sort’ defined but not used
swensonsort/sort.h:2175: warning: ‘proj2_swenson_grail_sort_dyn_buffer’ defined but not used
swensonsort/sort.h:2236: warning: ‘proj2_swenson_rec_stable_sort’ defined but not used
mpicc -I../../utils/Random123/include -g -Wall -std=c99 -fopenmp -fpic -O3 -c -o bitonic.o bitonic.c
mpicc -I../../utils/Random123/include -g -Wall -std

In [6]:
mpirun  -f ${PBS_NODEFILE} -n ${PBS_NP} ./test_proj2 400000 400000 1 0 1 0 0 0 

[0] ./test_proj2 minKeys 400000 maxKeys 400000 mult 2 seed 0 uniform size 0 uniform distribution 0 partially sorted 0
[0] Testing numKeysLocal 620439, numKeysGlobal 68490372, total bytes 547922976
[0] Tested numKeysLocal 620439, numKeysGlobal 68490372, total bytes 547922976: average bandwidth 2.717913e+09
[0] Harmonic average bandwidth: 2.717913e+09


In [57]:
hpcstruct ./test_proj2
mpirun  -f ${PBS_NODEFILE} -n ${PBS_NP} hpcrun -o hpctoolkit-test_proj2-measurements-singleDist -t ./test_proj2 400000 400000 32 0 1 0 0 0 

[0] ./test_proj2 minKeys 400000 maxKeys 400000 mult 32 seed 0 uniform size 0 uniform distribution 0 partially sorted 0
[0] Testing numKeysLocal 620439, numKeysGlobal 16310081, total bytes 130480648
[0] Tested numKeysLocal 620439, numKeysGlobal 16310081, total bytes 130480648: average bandwidth 9.504231e+08
[0] Harmonic average bandwidth: 9.504231e+08


In [59]:
# initial
# hpcprof -S test_proj2.hpcstruct hpctoolkit-test_proj2-measurements-129587.ice-sched.pace.gatech.edu 
# create subcomms first
# hpcprof -S test_proj2.hpcstruct hpctoolkit-test_proj2-measurements-130036.ice-sched.pace.gatech.edu
# merge with O(m+n)
hpcprof -S test_proj2.hpcstruct hpctoolkit-test_proj2-measurements-singleDist

msg: STRUCTURE: /nv/coc-ice/zjiang333/cse6230-hw/projects/2-sorting/test_proj2
msg: Line map : /nv/coc-ice/tisaac3/opt/pace-ice/hpctoolkit/lib/hpctoolkit/ext-libs/libmonitor.so.0.0.0
msg: Line map : /nv/coc-ice/zjiang333/cse6230-hw/projects/2-sorting/libproj2.so
msg: Line map : /nv/usr-local-rhel6.7/pacerepov1/intel/compiler/16.0/compilers_and_libraries_2016.0.109/linux/mpi/intel64/lib/libmpifort.so.12.0
msg: Line map : /nv/usr-local-rhel6.7/pacerepov1/intel/compiler/16.0/compilers_and_libraries_2016.0.109/linux/mpi/intel64/lib/release_mt/libmpi.so.12.0
msg: Line map : /lib64/libpthread-2.12.so
msg: Line map : /lib64/libc-2.12.so
msg: Line map : /lib64/ld-2.12.so
msg: Populating Experiment database: /nv/coc-ice/zjiang333/cse6230-hw/projects/2-sorting/hpctoolkit-test_proj2-database-singleDist


In [8]:
for uniform_size in 0 1; do
for uniform_keys in 0 1; do
for partially_sorted in 0 1 -1; do
mpirun  -f ${PBS_NODEFILE} -n ${PBS_NP} ./test_proj2 160 400000 32 0 5 ${uniform_size} ${uniform_keys} ${partially_sorted} 
done
done
done
  

[0] ./test_proj2 minKeys 160 maxKeys 400000 mult 32 seed 0 uniform size 0 uniform distribution 0 partially sorted 0
[0] Testing numKeysLocal 197, numKeysGlobal 26264, total bytes 210112
[0] Tested numKeysLocal 197, numKeysGlobal 26264, total bytes 210112: average bandwidth 1.426456e+08
[0] Testing numKeysLocal 7305, numKeysGlobal 867954, total bytes 6943632
[0] Tested numKeysLocal 7305, numKeysGlobal 867954, total bytes 6943632: average bandwidth 2.085459e+09
[0] Testing numKeysLocal 299023, numKeysGlobal 28079697, total bytes 224637576
[0] Tested numKeysLocal 299023, numKeysGlobal 28079697, total bytes 224637576: average bandwidth 2.644479e+09
[0] Harmonic average bandwidth: 3.812894e+08
[0] ./test_proj2 minKeys 160 maxKeys 400000 mult 32 seed 0 uniform size 0 uniform distribution 0 partially sorted 1
[0] Testing numKeysLocal 197, numKeysGlobal 26264, total bytes 210112
[0] Tested numKeysLocal 197, numKeysGlobal 26264, total bytes 210112: average bandwidth 1.434772e+08
[0] Testing num

[0] Tested numKeysLocal 163840, numKeysGlobal 18350080, total bytes 146800640: average bandwidth 4.095225e+09
[0] Harmonic average bandwidth: 3.530207e+08


In [5]:
hpcprof -S test_proj2.hpcstruct hpctoolkit-test_proj2-measurements-*.ice-sched.pace.gatech.edu

msg: Directory 'hpctoolkit-test_proj2-database-129279.ice-sched.pace.gatech.edu' already exists. Trying 'hpctoolkit-test_proj2-database-129279.ice-sched.pace.gatech.edu-129279.ice-sched.pace.gatech.edu'
msg: Created directory: hpctoolkit-test_proj2-database-129279.ice-sched.pace.gatech.edu-129279.ice-sched.pace.gatech.edu
msg: STRUCTURE: /nv/coc-ice/zjiang333/cse6230-hw/projects/2-sorting/test_proj2
msg: Line map : /nv/coc-ice/tisaac3/opt/pace-ice/hpctoolkit/lib/hpctoolkit/ext-libs/libmonitor.so.0.0.0
msg: Line map : /nv/coc-ice/zjiang333/cse6230-hw/projects/2-sorting/libproj2.so
msg: Line map : /nv/usr-local-rhel6.7/pacerepov1/intel/compiler/16.0/compilers_and_libraries_2016.0.109/linux/mpi/intel64/lib/release_mt/libmpi.so.12.0
msg: Line map : /lib64/libpthread-2.12.so
msg: Line map : /lib64/libc-2.12.so
msg: Line map : /lib64/ld-2.12.so
msg: Populating Experiment database: /nv/coc-ice/zjiang333/cse6230-hw/projects/2-sorting/hpctoolkit-test_proj2-database-129279.ice-sched.pace.gatech.

## Grading

- 0-4 points for hassle-free usage: maximized if the python script made from the notebook runs the first time.
    * Points lost if we have to figure out how to reproduce your reported results.
- 0-6 points for correctness:
    * Whether the notebook runs to
      completion (it will abort if a list of keys is not properly sorted).
    * You lose half the points if your code is not correct; subsequent points
      can be lost for poor code organization.
- 0-2 Points for your prediction and the reasoning that goes into it
- 0-8 Points for the notebook:
    * 0-2 points for how well the notebook tracks your `git` history: did we find the commits
      used to generate the entries?  Is there an entry for all the major
      aspects of your development?
    * 0-3 points for your profiling evidence: is it present?  Does it seem to
      indicate what you say it indicates?
    * 0-3 points for your planning: do your proposed code changes follow
      logically from the evidence?
- **1 Bonus point** for the closest prediction to the actual highest bandwidth achieved
- **1 Bonus point** for having versatile performance: if your code achieves bandwidths within 50% of the highest bandwidth achieved on that test for at least 50% of tests.
      
## Prediction

Predict, without going under, the highest bandwidth on any individual test that any student will achieve on this assignment.  Assume only the CPUs are used.  Justify your prediction.

The MPI_Bcast call sends one pivot (8 bytes) to all other processes. The time used is $t_1 = (\lambda + 8g) \lceil log_2 28 \rceil + (\lambda + 8g) \lceil log_2 14 \rceil + (\lambda + 8g) \lceil log_2 6 \rceil + (\lambda + 8g) \lceil log_2 3 \rceil + (\lambda + 8g) \lceil log_2 2 \rceil = 15(\lambda + 8g)$.  
The MPI_Isend and the MPI_Recv could overlap in best case. Denoting the number of keys to sort in total as N, the total time spent is: $t_2 = (\lambda + g\frac{8N}{2 \times 28}) + (\lambda + g\frac{8N}{4 \times 28} )+ (\lambda + g \frac{8N}{8 \times 28}) + (\lambda + g\frac{8N}{16 \times 28})  + (\lambda + g\frac{8N}{32 \times 28}) =5\lambda + \frac{8gN}{28} \times \frac{31}{32}$.  
while $\lambda << gN$, the predicted bandwidth $\approx$:  
$\frac{8N}{t_1 + t_2} = \frac{8N}{15(\lambda + 8g) + 5\lambda + \frac{8gN}{28} \times \frac{31}{32}} \lt \frac{8N}{\frac{8gN}{28} \times \frac{31}{32}} \approx \frac{28}{g} \approx 7.86 \times 10^{11} $ bytes/second

  
## Notebook

Please put your notebook documenting your measurements, thought processes, models, etc. from your work on this project

### Initial Perf Analysis:  
PS:   
1. Due to unknown reason (maybe the large netwark delay on VPN), mac cannot screenshot the window forwarded through ssh -XY. Photos of hpcvtraceviewer were taken for display. 
2. All hpc profiling and testing were done in single node with 28 processors setting. All version of modification were tested with `./test_proj2 400000 400000 32 0 1 0 0 0'`. Harmonic average bandwidth of provided loop test are also provided.
3. Modifications were made and evaluated incrementally instead of individually. 
  
I first profiled it with `hpcrun -t` and visualize the result with `hpctraceviewer`. The result is as follows:  
![image1](img/img1.jpg)
  
As we can see, the initial implementation of the quicksort had lots of drawbacks. Firstly, It is not efficient to split the Communications at every recursive call. Secondly, it sort the combined array up front and doesn't use the fact that it's concatenated by two sorted array, which takes extra time to sort local array. Thirdly, some ranks take longer to sort locally because the computation load is unbalanced. Lastly, the redistribution of the array is done at every recursive quicksort call. However, it is only needed at the end of quicksort.

### Modification and Analysis: 
**1. split and store subcomms at start**  
commit: `create all comms required at start, tested `  
To solve the first issue, I modified the code so it creates sub communications when creating the sorter and cache the array of pointers in the sorter struct. Then, instead of spliting at the end of every recursive call, it access the preassaigned subcomm depending on the call depth. The profiling result is:  
![image2](img/img2.jpg)
  
Compared to baseline, local sort takes up larger percent of time, because there is no overhead for spliting communications at the end of every recursive call. The test results are:
for `./test_proj2 400000 400000 32 0 1 0 0 0` 
The bandwidth goes from 1.845820e+08 (baseline) to 2.022028e+08.
Using the provided loop test for different conditions, 
Harmonic average bandwidth goes from 1.080773e+08 (baseline) to 1.572388e+08.   
**2. take advantage of two sorted arrays instead of quicksort to merge**  
commit: `merge two sorted array with O(m+n) method`  
To solve the second issue, an O(m+n) local scan merging algorithm is used when we merge two sorted half. The local quicksort (O(nlogn)) is only called when no sorting has been done (at depth 0). The profiling result is:   
![image3](img/img3.jpg)
  
Obviously, local sort was called much less frequently and was only called near the start. Instead, linear time local merge was used. This is especailly beneficial for ranks with large arrays to merge. The test results are:  
for `./test_proj2 400000 400000 32 0 1 0 0 0` :
The bandwidth goes from 2.022028e+08 (modification 1) to 8.386194e+08.  
Using the provided loop test for different conditions,   
Harmonic average bandwidth goes from 1.572388e+08 to 7.776744e+08.   
Based on the profiling and test results, this modification gives us the largest boost in performance.  
**3. better pivot to balance load**  
commit: `different pivot selection strategy`  
Although mod2 provides a large boost, quicksort division is still unbalanced because we are using the median of one array as our estimated pivot, which could be very biased. To give a better estimate, I adopted the median of $p^2$ numbers, which are samples from p threads with p numbers each, sampled at uniform intervals. i.e., for each processor, it takes p points, at 100/p %, 200/p %, etc. The profiling result is:   
![image4](img/img4.jpg)
Compared to mod2, sorting load is much more balanced here. However, it create $O(2size^2log size)$ overhead (compared to O(1)) everytime we find a pivot, which is not worth it when sorting short arrays. The test results are:    
for `./test_proj2 400000 400000 32 0 1 0 0 0` :  
The bandwidth goes from 8.386194e+08 to 8.965145e+08.   
Using the provided loop test for different conditions,   
Harmonic average bandwidth goes from 7.776744e+08 to 7.665905e+08.    
Again, this reiterates that overhead to select the pivot in sorting small arrays cannot be ignored.      
**4. reditribute once at the end of recursive calls**   
commit: `redistribute array only at the end of each recursive call`  
To tackle the unneccessary redistribution, I changed the code to redistribute only at the end of all recursive calls. **This is also the last modification I made. Therefore, performance on four nodes were also evaluated.** The profiling result is:  
![image5](img/img5.jpg)  
Now the redistribution only happens at the end of recursive calls, which slight improves the performance.  
for `./test_proj2 400000 400000 32 0 1 0 0 0` :    
The bandwidth goes from 8.965145e+08 to 9.650866e+08. On four nodes, it's 2.710517e+09.  
Using the provided loop test for different conditions,   
Harmonic average bandwidth goes from 7.665905e+08 to 1.011683e+09. On four nodes, it's 3.530207e+08. This may indicates that higher number of processors brings large overhead for sorting short arrays.  
PS: weird MPI errors happen at times pretty randomly, although the program still finishs correctly with these errors. 
